# Module 2 Part 1.1 EDA with SQL

In [13]:
import pandas as pd
import sqlite3
from prettytable import PrettyTable
import csv
import numpy as np

### Connect to the database
#### Let us first load the SQL extension and establish a connection with the database

In [14]:
con = sqlite3.connect('my_data1.db')
cur = con.cursor()

# Drop the table if it exists BEFORE importing the CSV
drop_table = '''DROP TABLE IF EXISTS SPACEXTBL;'''
cur.execute(drop_table)

# Load data from CSV
file = '/Users/stalinjaquez/Desktop/VSCode/IBM Data Certificate/Course11_Applied_Data_Science_Capstone/Spacex.csv'
df = pd.read_csv(file)

# Write data to the table
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method='multi')

# Create the new table filtering non-null dates
table = '''SELECT * FROM SPACEXTBL WHERE Date IS NOT NULL;'''
cur.execute(table)

#### Columns # 
- 'Date' 
- 'Time (UTC)'
- 'Booster_Version'
- 'Launch_Site'
- 'Payload' 
- 'PAYLOAD_MASS__KG_'
- 'Orbit'
- 'Customer'
- 'Mission_Outcome'
- 'Landing_Outcome'

#### Task 1: Display the names of the unique launch sites in the space mission.

In [15]:
task_1 = '''SELECT DISTINCT Launch_Site FROM SPACEXTBL'''
cur.execute(task_1)
results = cur.fetchall()
columns =[]
for column in cur.description:
    columns.append(column[0])

table = PrettyTable(columns)
for row in results:
    table.add_row(row)

print(table)

+--------------+
| Launch_Site  |
+--------------+
| CCAFS LC-40  |
| VAFB SLC-4E  |
|  KSC LC-39A  |
| CCAFS SLC-40 |
+--------------+


#### Task 2: Display 5 records where launch sites begin with the string 'CCA'.

In [21]:
task_2 = '''SELECT * 
FROM SPACEXTBL 
WHERE Launch_Site LIKE 'CCA%' LIMIT 5;'''
cur.execute(task_2)
results = cur.fetchall()

columns = []
table = []
for column in cur.description:
    columns.append(column[0])

table = PrettyTable(columns)
for row in results:
    table.add_row(row)

print(table)

+------------+------------+-----------------+-------------+---------------------------------------------------------------+-------------------+-----------+-----------------+-----------------+---------------------+
|    Date    | Time (UTC) | Booster_Version | Launch_Site |                            Payload                            | PAYLOAD_MASS__KG_ |   Orbit   |     Customer    | Mission_Outcome |   Landing_Outcome   |
+------------+------------+-----------------+-------------+---------------------------------------------------------------+-------------------+-----------+-----------------+-----------------+---------------------+
| 2010-06-04 |  18:45:00  |  F9 v1.0  B0003 | CCAFS LC-40 |              Dragon Spacecraft Qualification Unit             |         0         |    LEO    |      SpaceX     |     Success     | Failure (parachute) |
| 2010-12-08 |  15:43:00  |  F9 v1.0  B0004 | CCAFS LC-40 | Dragon demo flight C1, two CubeSats, barrel of Brouere cheese |         0         | 

#### Task 3: Display the total payload mass carried by boosters launched by NASA (CRS).

In [23]:
task_3 = '''SELECT CUSTOMER, SUM(PAYLOAD_MASS__KG_) AS SUM_PAYLOAD_MASS_KG 
FROM SPACEXTBL 
WHERE CUSTOMER = 'NASA (CRS)';'''
cur.execute(task_3)
results = cur.fetchall()

columns = []
table = []
for column in cur.description:
    columns.append(column[0])

table = PrettyTable(columns)
for row in results:
    table.add_row(row)
print(table)

+------------+---------------------+
|  Customer  | SUM_PAYLOAD_MASS_KG |
+------------+---------------------+
| NASA (CRS) |        45596        |
+------------+---------------------+


#### Task 4: Display average payload mass carried by booster version F9 v1.1. 

In [24]:
task_4 = '''SELECT Booster_Version, AVG(PAYLOAD_MASS__KG_) AS AVG_PAYLOAD_MASS__KG 
FROM SPACEXTBL 
WHERE Booster_Version = 'F9 v1.1' '''
cur.execute(task_4)
results = cur.fetchall()

columns = []
table = []
for column in cur.description:
    columns.append(column[0])

table = PrettyTable(columns)
for row in results:
    table.add_row(row)

print(table)

+-----------------+----------------------+
| Booster_Version | AVG_PAYLOAD_MASS__KG |
+-----------------+----------------------+
|     F9 v1.1     |        2928.4        |
+-----------------+----------------------+


#### Task 5: List the date when the first succesful landing outcome in ground pad was acheived.

In [26]:
task_5 = '''SELECT Landing_Outcome, MIN(DATE) AS First_Ground_Pad_Landing_Date 
FROM SPACEXTBL 
WHERE Landing_Outcome = 'Success (ground pad)' '''
cur.execute(task_5)
results = cur.fetchall()

columns = []
table = []
for column in cur.description:
    columns.append(column[0])

table = PrettyTable(columns)
for row in results:
    table.add_row(row)

print(table)

+----------------------+-------------------------------+
|   Landing_Outcome    | First_Ground_Pad_Landing_Date |
+----------------------+-------------------------------+
| Success (ground pad) |           2015-12-22          |
+----------------------+-------------------------------+


#### Task 6: List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000.

In [27]:
task_6 = '''SELECT DISTINCT(Booster_Version),PAYLOAD_MASS__KG_ ,Landing_Outcome 
FROM SPACEXTBL WHERE Landing_Outcome = 'Success (drone ship)' 
AND PAYLOAD_MASS__KG_ > 4000 
AND PAYLOAD_MASS__KG_ < 6000 '''
cur.execute(task_6)
results = cur.fetchall()

columns = []
table = []
for column in cur.description:
    columns.append(column[0])

table = PrettyTable(columns)
for row in results:
    table.add_row(row)

print(table)

+-----------------+-------------------+----------------------+
| Booster_Version | PAYLOAD_MASS__KG_ |   Landing_Outcome    |
+-----------------+-------------------+----------------------+
|   F9 FT B1022   |        4696       | Success (drone ship) |
|   F9 FT B1026   |        4600       | Success (drone ship) |
|  F9 FT  B1021.2 |        5300       | Success (drone ship) |
|  F9 FT  B1031.2 |        5200       | Success (drone ship) |
+-----------------+-------------------+----------------------+


#### Task 7: List the total number of successful and failure mission outcomes. 

In [28]:
task_7 = '''SELECT Mission_Outcome, COUNT(*) AS Count
FROM SPACEXTBL
GROUP BY Mission_Outcome;
'''
cur.execute(task_7)
results = cur.fetchall()

columns = []
table = []
for column in cur.description:
    columns.append(column[0])

table = PrettyTable(columns)
for row in results:
    table.add_row(row)

print(table)

+----------------------------------+-------+
|         Mission_Outcome          | Count |
+----------------------------------+-------+
|       Failure (in flight)        |   1   |
|             Success              |   98  |
|             Success              |   1   |
| Success (payload status unclear) |   1   |
+----------------------------------+-------+


#### Task 8: List the names of the booster_versions which have carried the maximum payload mass. Use a subquery.

In [29]:
task_8 = '''SELECT Booster_Version, PAYLOAD_MASS__KG_
FROM SPACEXTBL
WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTBL);
'''
cur.execute(task_8)
results = cur.fetchall()

columns = []
table = []
for column in cur.description:
    columns.append(column[0])

table = PrettyTable(columns)
for row in results:
    table.add_row(row)

print(table)

+-----------------+-------------------+
| Booster_Version | PAYLOAD_MASS__KG_ |
+-----------------+-------------------+
|  F9 B5 B1048.4  |       15600       |
|  F9 B5 B1049.4  |       15600       |
|  F9 B5 B1051.3  |       15600       |
|  F9 B5 B1056.4  |       15600       |
|  F9 B5 B1048.5  |       15600       |
|  F9 B5 B1051.4  |       15600       |
|  F9 B5 B1049.5  |       15600       |
|  F9 B5 B1060.2  |       15600       |
|  F9 B5 B1058.3  |       15600       |
|  F9 B5 B1051.6  |       15600       |
|  F9 B5 B1060.3  |       15600       |
|  F9 B5 B1049.7  |       15600       |
+-----------------+-------------------+


#### Task 9: List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

In [30]:
task_9 = '''SELECT strftime('%m', Date) AS Month, strftime('%Y', Date) AS Year, Booster_Version, Launch_Site, Landing_Outcome
FROM SPACEXTBL
WHERE Landing_Outcome LIKE 'Failure (drone ship)'
AND Year = '2015';'''

cur.execute(task_9)
results = cur.fetchall()

columns = []
table = []
for column in cur.description:
    columns.append(column[0])

table = PrettyTable(columns)
for row in results:
    table.add_row(row)

print(table)

+-------+------+-----------------+-------------+----------------------+
| Month | Year | Booster_Version | Launch_Site |   Landing_Outcome    |
+-------+------+-----------------+-------------+----------------------+
|   01  | 2015 |  F9 v1.1 B1012  | CCAFS LC-40 | Failure (drone ship) |
|   04  | 2015 |  F9 v1.1 B1015  | CCAFS LC-40 | Failure (drone ship) |
+-------+------+-----------------+-------------+----------------------+


#### Task 10: Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order. 

In [31]:
task_10 = '''SELECT Landing_Outcome, COUNT(*) AS Outcome_Count
             FROM SPACEXTBL
             WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'
             GROUP BY Landing_Outcome
             ORDER BY Outcome_Count DESC;'''
cur.execute(task_10)
results = cur.fetchall()

columns = []
table = []
for column in cur.description:
    columns.append(column[0])

table = PrettyTable(columns)
for row in results:
    table.add_row(row)

print(table)




+------------------------+---------------+
|    Landing_Outcome     | Outcome_Count |
+------------------------+---------------+
|       No attempt       |       10      |
|  Success (drone ship)  |       5       |
|  Failure (drone ship)  |       5       |
|  Success (ground pad)  |       3       |
|   Controlled (ocean)   |       3       |
|  Uncontrolled (ocean)  |       2       |
|  Failure (parachute)   |       2       |
| Precluded (drone ship) |       1       |
+------------------------+---------------+
